In [ ]:
from transformers import ASTModel, ASTConfig
import torch
import torch.nn as nn

In [ ]:
config = ASTConfig(hidden_size=384, num_attention_heads=6, intermediate_size=1536)
model = ASTModel(config=config)

config_tiny = ASTConfig(hidden_size=192, num_attention_heads=3, intermediate_size=769)
model_tiny = ASTModel(config=config)

In [ ]:
input = torch.rand((1,1024,128))
output = model(input)

In [ ]:
class ASTSmallDistill(nn.Module):
    
    def __init__(self, ):
    
        config = ASTConfig(hidden_size=384, num_attention_heads=6, intermediate_size=1536)
        model = ASTModel(config=config)
        pass

In [ ]:
from types import SimpleNamespace

In [ ]:
test = SimpleNamespace(**{"test": 5})

In [ ]:
SimpleNamespace().__dict__

In [ ]:
from lumen_irmas import ASTPretrained

In [ ]:
model = ASTPretrained(11,0.5)

In [ ]:
getattr(model.base_model, "encoder")

In [ ]:
from lumen_irmas.modeling.models import freeze

In [ ]:
import torch
import torch.nn as nn
from lumen_irmas import ASTPretrained

In [ ]:
import torch
import torch.nn as nn
from lumen_irmas import ASTPretrained
num_models = 2
batch_size = 2
out_features = 11
models = [ASTPretrained(out_features, 0.5).eval().to("cuda:0") for _ in range(num_models)]

In [ ]:
model = ASTPretrained(11,0.5).eval().to("cuda:1")

In [ ]:
!nvidia-smi

In [ ]:
data = data.to("cuda:1")
model(data)

In [ ]:
data = torch.randn(batch_size, 1024, 128).to("cuda:1")
data = data.to("cuda:0")


# ------------------------------------
# using torch.func (as of PyTorch 2.0)
# ------------------------------------
import copy

# Construct a version of the model with no memory by putting the Tensors on
# the meta device.
base_model = copy.deepcopy(models[0]).to('meta')

params, buffers = torch.func.stack_module_state(models)

# It is possible to vmap directly over torch.func.functional_call,
# but wrapping it in a function makes it clearer what is going on.
def call_single_model(params, buffers, data):
    return torch.func.functional_call(base_model, (params, buffers), (data,))

output = torch.vmap(call_single_model, (0, 0, None))(params, buffers, data)
output

In [ ]:
output = torch.vmap(call_single_model, (0, 0, None))(params, buffers, data)
assert output.shape == (num_models, batch_size, out_features)
output

In [ ]:
output = [model(data) for model in models]

In [ ]:
torch.Tensor(output)

In [ ]:
torch.mean(output, 0)

In [ ]:
data = torch.randn(8, 1024, 128).to("cuda:1")

In [ ]:
torch.vmap(call_single_model, (0, 0, None))(params, buffers, data)

In [ ]:
!nvidia-smi

In [1]:
import torch.nn as nn
import torch
from lumen_irmas import ASTPretrained

In [2]:
class Ensemble(nn.Module):
    def __init__(self, models):
        super(Ensemble, self).__init__()
        self.models = [nn.DataParallel(model).to("cuda") for model in models]

    def forward(self, x):
        predictions = []
        for model in self.models:
            predictions.append(model(x))
        return torch.mean(torch.stack(predictions), dim=0)

In [3]:
teachers = Ensemble([ASTPretrained(11,0.5) for _ in range(2)])

Some weights of the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 were not used when initializing ASTModel: ['classifier.layernorm.weight', 'classifier.dense.weight', 'classifier.layernorm.bias', 'classifier.dense.bias']
- This IS expected if you are initializing ASTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ASTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 were not used when initializing ASTModel: ['classifier.layernorm.weight', 'classifier.dense.weight', 'classifier.layernorm.bias', 'classifier.dense.bias']
- This IS expected if you are initializing ASTModel from the checkpoint of a mo

In [4]:
!nvidia-smi

Mon Apr 10 19:15:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 495.46       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:00:08.0 Off |                  N/A |
| 28%   24C    P2    56W / 250W |   1199MiB / 11178MiB |     16%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:00:09.0 Off |                  N/A |
| 20%   

In [5]:
data = torch.rand(4,1024,128).to("cuda")
output = teachers(data)

In [6]:
!nvidia-smi

Mon Apr 10 19:15:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 495.46       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:00:08.0 Off |                  N/A |
| 28%   25C    P2    63W / 250W |   7725MiB / 11178MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:00:09.0 Off |                  N/A |
| 20%   

In [1]:
from transformers import ASTConfig, ASTModel

In [3]:
config = ASTConfig(hidden_size=768, num_attention_heads=6, intermediate_size=768*2)
ASTModel(config=config)

ASTModel(
  (embeddings): ASTEmbeddings(
    (patch_embeddings): ASTPatchEmbeddings(
      (projection): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ASTEncoder(
    (layer): ModuleList(
      (0-11): 12 x ASTLayer(
        (attention): ASTAttention(
          (attention): ASTSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): ASTSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ASTIntermediate(
          (dense): Linear(in_features=768, out_features=1536, bias=True)
          (intermediate_act_fn): GELUActivation(

In [3]:
import torch
import torch.nn as nn
from lumen_irmas.modeling.models import Ensemble, StudentAST, freeze

In [4]:
data = torch.rand(1,1024,128)

In [3]:
student = StudentAST(n_classes=11, hidden_size=192, num_heads=3)

teacher1 = StudentAST(n_classes=11, hidden_size=192, num_heads=3)
teacher2 = StudentAST(n_classes=11, hidden_size=192, num_heads=3)
teachers = Ensemble([teacher1, teacher2])

In [4]:
student_output = student(data)
teachers_output = teachers(data)

In [5]:
teachers_output

tensor([[ 0.5243, -0.0507,  0.0611,  0.2231, -0.1932, -0.2000, -0.0975,  0.4029,
          0.2513, -0.2964,  0.2300]])

In [6]:
from lumen_irmas.modeling.loss import DistillationLoss, FocalLoss

In [7]:
loss_fn = DistillationLoss(teachers, FocalLoss(alpha=-1))

In [8]:
targets = torch.tensor([0,0,0,1,1,1,0,0,0,0,1]).unsqueeze(0).float()

In [9]:
loss_fn(student_output, teachers_output, targets)

(tensor(0.2674, grad_fn=<MeanBackward0>),
 tensor(0.3272, grad_fn=<MeanBackward0>))

In [8]:
from transformers import ASTModel, ASTConfig

In [9]:
config = ASTConfig(hidden_size=384,
                    num_attention_heads=6, intermediate_size=384*2)
model = ASTModel(config=config)

In [15]:
model(data)[1][:, 0].shape

torch.Size([1])